# Example script to identify PPAs

_Run on daily z500 field clipped to (xn,xx), (yn,yx) - gridded ERA5 z500 reanalysis can be downloaded from https://climexp.knmi.nl_

In [2]:
import xarray as xr, numpy as np, geopandas as gpd, regionmask
import glob, re, os

from datetime import datetime
from  IPython.display import clear_output

import PPA_functions as PPA

xn,xx,yn,yx = [-150.0,-50.0,40.0,75.0]

ModuleNotFoundError: No module named 'PPA_functions'

In [2]:
# set thresholds to identify PPAs
sigma_threshold = 1. # identify as PPA if anomaly is more than sigma_threshold standard deviations
duration_threshold = 5 # days
area_threshold = 100000. # units = km^2

# Identify PPAs

_Run on full gridded z500 field clipped to (xn,xx), (yn,yx)_

In [3]:
# specify filename
z500_fnm = "era5_z500_daily_-150--50E_40-75N.nc"

In [4]:
# get grid cell_area for checking extent of PPAs
! module load cdo; cdo gridarea $z500_fnm gridarea_era5.nc
clear_output(wait = False)
gridarea = PPA.wrap_lon(xr.open_dataset("gridarea_era5.nc"))

In [5]:
# load z500 data, convert to calendar without leap years
da = PPA.wrap_lon(xr.open_dataset(z500_fnm)).z500.load()
da = da.convert_calendar("noleap", align_on = "year")

# detrend and save results
z500_det = PPA.detrend_z500(da).rename(lat = "latitude", lon = "longitude")

In [6]:
# get climatology (used to identify significant anomalies) and smooth
clim_mean, clim_sd = PPA.sm_climatology(z500_det)
clim_mean, clim_sd = [xr.Dataset(data_vars = {"z" : da.rolling(time = 5, center = True).mean()}) for da in [clim_mean, clim_sd]]

In [7]:
# loop over all years, get PPA anomalies & save as indvidual yearly files before combining into single dataframe
for year in np.unique(da.time.dt.year.values)[:12]:
    
    print(year)
    y_fnm = "tmp/ppa-mask_"+str(year)+".nc"
    if os.path.exists(y_fnm): continue

    # select year of interest
    da_y = z500_det.sel(time = str(year))

    # if year is incomplete, print warning & skip
    if not da_y.shape == clim_mean.z.shape:
        print("Incomplete year: ", str(year))
        continue
    
    # compute annual anomalies
    da_anom = xr.Dataset(data_vars = {"z" : da_y - clim_mean.z.values})

    # identify PPAs (must exceed specified SD threshold above mean)
    anom_mask, ppa_mask = PPA.mask_Z500_anom_by_threshold(da_anom, clim_sd, sigma_threshold = sigma_threshold)

    # mask by minimum duration criterion
    ppa_mask = PPA.mask_array_by_duration(ppa_mask, duration_threshold)

    # mask by minimum area criterion
    ppa_mask = PPA.mask_by_area(ppa_mask, gridarea, area_threshold)

    # mask by duration again to ensure masking by area hasn't led to events shorter than duration_threshold
    ppa_mask = PPA.mask_array_by_duration(ppa_mask, duration_threshold)
    
    # convert PPA mask back to DataArray & add to list
    ppa_mask = xr.DataArray(data = ppa_mask, dims = ["time", "latitude", "longitude"], name = "z", coords = dict(da_y.coords))
    ppa_mask.to_netcdf(y_fnm)
    

    clear_output(wait = False)

In [3]:
# compile all years into a single dataarray
ppa_masks = xr.concat([xr.open_dataset(fnm) for fnm in sorted(glob.glob("tmp/ppa-mask_*.nc"))], "time")

## Compute PPA time series in each region

In [4]:
# load shapefile and create regionmask
sf = gpd.read_file("sf").to_crs(epsg = 4326)
rm = regionmask.mask_3D_geopandas(sf, ppa_masks.longitude, ppa_masks.latitude)

ppa_ts = ppa_masks.z.where(rm).mean(["latitude", "longitude"])

In [5]:
# get monthly and (fire-season only) annual time series
ppa_m = ppa_ts.resample(time = "MS").sum()
ppa_y = ppa_m.sel(time = ppa_m.time.dt.month.isin([4,5,6,7,8,9,10])).resample(time = "YS").sum()